In [11]:
from IPython.core.display import display, HTML
display(HTML("<style>.output_area pre { white-space: pre-wrap; word-wrap: break-word; }</style>"))


/var/folders/3p/s2hzq46x79zbl3zb0t91q2y00000gn/T/ipykernel_7870/4045732499.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [1]:
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions


# client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet",
#                                     persist_directory="db/"
#                                 ))

client = chromadb.PersistentClient(path="db")

code sets up a client to interact with ChromaDB Database
directory for stopring database files db/


In [4]:
embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

/Users/jacobschorr/Documents/RandyCode/fresh_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
collection = client.get_or_create_collection(name="Courses", embedding_function=embedding_fn)

In [7]:
computer_science = """
A Computer Science major focuses on the study of algorithms, programming, software development, and computational theory. Students learn various programming languages, data structures, and system architectures to develop efficient and scalable applications. The major also covers areas like artificial intelligence, cybersecurity, databases, and networking, preparing students for diverse tech careers. Hands-on projects and internships help build problem-solving skills and real-world experience in software engineering, machine learning, or web development. Graduates can pursue roles such as software developer, data scientist, or IT specialist in industries ranging from finance to healthcare and technology.
"""

history= """
A History major explores past events, societies, cultures, and significant figures to understand how they shape the present and future. Students analyze primary and secondary sources, learning to think critically, interpret evidence, and construct well-supported arguments. The major covers various periods and regions, from ancient civilizations to modern global conflicts, often incorporating political, economic, and social perspectives. Research projects and discussions help develop strong writing, analytical, and communication skills. Graduates can pursue careers in education, law, public policy, museum curation, or journalism, leveraging their ability to synthesize complex information and contextualize historical trends.

"""

In [14]:
import csv
#DONT NEED TO RERUN
def read_csv_to_list(file_path):
    documents = []
    metadata = []
    ids = []
    i=0
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for i, row in enumerate(reader):
            if i>=1:
                documents.append(', '.join(row))  # Convert row to a string
                if len(row) > 1:
                    course_code = row[1].split(' -')[0]  # Strip everything including and after '-'
                    metadata.append({'course_code': course_code,'subject_area': row[0] })  # Extract cleaned course 
                ids.append(f"doc_{i}")  # Assign a unique ID to each document
            i+=1
    return documents, metadata, ids

# Example usage
file_path = 'subject_courses.csv'  # Replace with your actual file path
course_documents, course_metadata, ids = read_csv_to_list(file_path)



In [16]:
course_metadata

[{'course_code': 'CORE1010', 'subject_area': 'A&S College Core'},
 {'course_code': 'CORE1020', 'subject_area': 'A&S College Core'},
 {'course_code': 'CORE2500', 'subject_area': 'A&S College Core'},
 {'course_code': 'AADS1001',
  'subject_area': 'African American Diaspora Studies'},
 {'course_code': 'AADS1010',
  'subject_area': 'African American Diaspora Studies'},
 {'course_code': 'AADS1016',
  'subject_area': 'African American Diaspora Studies'},
 {'course_code': 'AADS1108',
  'subject_area': 'African American Diaspora Studies'},
 {'course_code': 'AADS1111',
  'subject_area': 'African American Diaspora Studies'},
 {'course_code': 'AADS1204',
  'subject_area': 'African American Diaspora Studies'},
 {'course_code': 'AADS1404',
  'subject_area': 'African American Diaspora Studies'},
 {'course_code': 'AADS1506',
  'subject_area': 'African American Diaspora Studies'},
 {'course_code': 'AADS1706',
  'subject_area': 'African American Diaspora Studies'},
 {'course_code': 'AADS1716',
  'subje

In [9]:
#DONT NEED TO RERUN
collection.upsert(documents=course_documents,
metadatas = course_metadata,
ids = ids,
)

KeyboardInterrupt: 

In [3]:
#DONT NEED TO RERUN
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
import uuid

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file and return a list of texts per page."""
    text_by_page = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text() or ""  # Extract text or set empty string if none
            text_by_page.append(text)
    return text_by_page

def chunk_text(text_list, chunk_size=1500, chunk_overlap=250):
    """Split text into overlapping chunks and store metadata for ChromaDB."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )

    documents = []
    metadatas = []
    ids = []

    for i, page_text in enumerate(text_list):
        chunks = text_splitter.split_text(page_text)
        for chunk in chunks:
            chunk_id = str(uuid.uuid4())  # Generate unique ID
            documents.append(chunk)
            metadatas.append({"page": i + 1})  # Store page number as metadata
            ids.append(chunk_id)
    
    return documents, metadatas, ids

def save_chunks_to_json(documents, metadatas, ids, output_file="chunks.json"):
    """Save extracted chunks to a JSON file."""
    chunk_data = [
        {"id": ids[i], "text": documents[i], "metadata": metadatas[i]}
        for i in range(len(documents))
    ]
    
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(chunk_data, f, indent=4)

# Example Usage
pdf_path = "Undergraduate_Catalog_2024-25.pdf"  # Update with actual PDF file
text_list = extract_text_from_pdf(pdf_path)
documents, metadatas, ids = chunk_text(text_list)

# Save chunks for reference or direct insertion into ChromaDB
save_chunks_to_json(documents, metadatas, ids)
print(f"Extracted {len(documents)} chunks. Saved to chunks.json.")


Extracted 1217 chunks. Saved to chunks.json.


In [ ]:
catalog_documents = documents
catalog_metadatas = metadatas
catalog_ids = ids


In [19]:
#DONT NEED TO RERUN
collection = client.get_or_create_collection(name="undergraduate_catalog")

# Load JSON and insert into ChromaDB
import json
with open("chunks.json", "r", encoding="utf-8") as f:
    chunk_data = json.load(f)

# Insert into ChromaDB
collection.upsert(
    ids=[chunk["id"] for chunk in chunk_data],
    documents=[chunk["text"] for chunk in chunk_data],
    metadatas=[chunk["metadata"] for chunk in chunk_data]
)


collection object is similar to creating a table in a traditional database
**Collection** is a logical grouping of data that share a context or purpose (one major in our case):
A collection contains
1. documents (text or data that you are storing)
2. metadata (slourse, date, tags etc)
3. Embeddings
4. IDs

In [6]:
print(collection.count())

# private endpoint if running http server (in that case, its fine):
print(client.list_collections())

3928
['Courses', 'Majors', 'undergraduate_catalog']


collection object is similar to creating a table in a traditional database
**Collection** is a logical grouping of data that share a context or purpose (one major in our case):
A collection contains
1. documents (text or data that you are storing)
2. metadata (slourse, date, tags etc)
3. Embeddings
4. IDs

In [7]:
collection = client.get_or_create_collection(name="Courses", embedding_function=embedding_fn)
results = collection.query(
    query_texts=["what requirements are there for computer science"], # Chroma will embed this for you
    n_results=8, # how many results to return
    include=[ 'documents','metadatas']
)
print(results)

{'ids': [['doc_938', 'doc_939', 'doc_922', 'doc_940', 'doc_906', 'doc_968', 'doc_942', 'doc_916']], 'embeddings': None, 'documents': [['Computer Science, CS3860 - Undergraduate Research, Open to qualified majors with consent of instructor and adviser. No more than 6 hours of CS 3860/3861/3862 may be counted towards the computer science major. Prerequisite: CS 3251. [1-3 each semester]', 'Computer Science, CS3861 - Undergraduate Research, CS 3861: Open to qualified majors with consent of instructor and adviser. No more than 6 hours of CS 3860/3861/3862 may be counted towards the computer science major. Prerequisite: CS 3251. [1-3 each semester]', 'Computer Science, CS2860 - Introduction to Undergraduate Research, Introduction to undergraduate research under the direction of Computer Science faculty. Students may take a maximum of 3 credit hours towards the degree. Not open to students who have earned credit for CS 3860, 3861, or 3251 without permission. Prerequisite: CS 1100 or CS 1101.

In [8]:
collection = client.get_or_create_collection(name="undergraduate_catalog")

In [9]:
results = collection.query(
    query_texts=["what requirements are there for computer science"], # Chroma will embed this for you
    n_results=8, # how many results to return
    include=[ 'documents','metadatas']
)
results

{'ids': [['919976fa-7461-43b0-916e-66123aab657a',
   'd5016419-b1e2-4bfe-9dd6-fef411dd6175',
   'ae28065a-7ce5-402c-9783-04b357c40003',
   'd3b60ff5-4d31-4148-ba29-88aa5a5a1cf2',
   'a3a578f2-d5e6-45fc-8e4c-2608a8babb20',
   '0f18c1ac-3533-4bee-8f0b-f3415e1c9a96',
   '736e21e6-c1b9-48d1-b591-19a1f671a942',
   '0e877979-3d45-4936-a14f-662332b8d8cc']],
 'embeddings': None,
 'documents': [['to complement their CS studies. Popular double majors include Mathematics, Economics, HOD and Physics. The\nsecond major in Computer Science is accessible to students outside of the School of Engineering. Second major\nrequirements can be found following the engineering specimen curriculum below.\nUndergraduate Honors. Honors in Computer Science provides recognition for select undergraduates who have\nexperienced advanced study in computer science. Students who have an overall GPA of 3.5 or better, a GPA of 3.5\nor better in computer science classes, and six hours of any combination of undergraduate re

In [15]:
summary = "This is a long text output that should wrap correctly when printed."
print(summary)


This is a long text output that should wrap correctly when printed.


import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)